Pseudo skeleton code for ACO algorithm applied to TSP (Mohsin,Thangavelu. (2021))

Introduction to the solution 
%Ant system for solving the traveling salesman problem:
<Input of matrix D of distances>
<Initialization of the parameters of the algorithm: α, β, Q, p, and τ0>
m=n %Number of ants is equal to the number of towns
For k=1:m
    <Allocate ant k in randomly chosen town>
End
<Select a conditionally shortest route T+ and calculate its length L+>;
%Main loop
For t=1:tma
 %For each ant:
    <Build a route Tk(t) according to rule (1)>
    <Calculate Lk(t), the length of the route Tk(t)>
    End
    If “Is the best solution found?” <Update T+ and L+> End
    For i=1:n
        For j=1:n %For each edge
            <Update pheromone trails according to rule (2)>
        End
    End
End

Sections
Read tsp file 
Create LIST with the cities and coordinates
create ARRAY with distances between cities (the number of distances is a combination nCr 323 distancias)

Questions:
Whats the answer format? And array with the rute example city1, city4, city5....cityn

In [1]:
##import libraries
import math
import pandas as pd
import numpy as np
import random

following code shows script to read the content of the tsp file and store it in a list and each index is a tuple that contains city-n, x,y
[( ),( ),( ),( ),....................,( )]
#from now on we are going to address the cities by their index in the list/Array/matrix instead of their city collum value to prevent confusions 
index[n-1]   city_n    coordinatesX     coordinatesY
0               1       11003.611100    42102.500000           

In [2]:
# Open input file
cities = [] #create array to store cities
file = open('dj38.tsp', 'r') #read file
lines = file.readlines() #read lines
for line in (lines[10:]): #loop over lines
    # Assumes the file format: <CityIndex> <X> <Y>
    city_index, x, y = map(float, line.strip().split()) #separate elements by blank space
    cities.append((city_index, x, y)) #creates a list with al the elements city, x, y

Following code shows the calculation of distances between the cities, visibility, and matrixes fill up 
distance_func: Function that calculates the distance between two nodes/points/Cities given two cities (i,j)
visibility_func: function that calculates the visibility between two nodes/points/cities given two cities(i,j)

distances: (D_ij) matrix that stores all distances between nodes
visibility: (η_ij) matrix that stores all visibilities between nodes
pheromones_t:  (𝜏_ij(t)) matrix that stores pheromone strenght (number of virtual pheromone on the edge i,j) between nodes at t time (iteration) changes with each iteration

In [3]:
def distance_func(city_i,city_j): #city i is current city while city j is the next city
    x1=cities[city_i][1] #identify x1 coordinate
    x2=cities[city_j][1] #identify x2 coordinate
    y1=cities[city_i][2] #identify y1 coordinate
    y2=cities[city_j][2] #identify y2 coordinate
    d=math.sqrt((x2-x1)**2+(y2-y1)**2) ##apply formula for the distance between two points
    d=round(d,2)
    return d

def visibility_func(city_i,city_j): #city i is current city while city j is the next city
    if distances[city_i][city_j]==0:
        nij=0
    else:
        nij=1/distance_func(city_i,city_j) #distances[city_i][city_j] ##apply formula for the distance between two points
        nij=round(nij,3)
    return nij

#matrixes declaration
distances = np.array([np.array([distance_func(i,j) for i in range(len(cities))]) for j in range(len(cities))])
visibilities = np.array([np.array([visibility_func(i,j) for i in range(len(cities))])for j in range(len(cities))])
pheromones_t = np.ones((len(cities),len(cities))) #𝜏_ij

following codes shows initial solution for ONE ant

In [12]:
#parameters
alpha = 1                                                                   #pheromone value(how likely is to take into account pheromone value)
beta = 1.1                                                                    #heuristic value(how likely is to take randomness into account?)
p = .95                                                                       #pheromone evaporation coefficient
n=len(cities)                                                                 
k=n                                                                           #number of ants given the number of cities
ants_routes=[]                                                                #list that stores the routes ants take in their trips

#Ant_k starts in city i (same value as ant) and then goes on to j and so on until it completes a route and then goes Ant_1....and so on until Ant_n
for ant_k in range (len(cities)):
      tabu_list_k = np.array([x for x in range (len(cities))])                      #list of unvisited cities for ant k
      i=ant_k                                                                       #starts at city 0
      tabu_list_k = np.delete(tabu_list_k,np.where(tabu_list_k==i))                 #deletes from the tabu list on this ant k the city they are visiting
      route_ant_k=[i]
      route_lenght=0                                                                       

      #next city decision
      while i not in tabu_list_k:                                                   #while ant at city (i) not in the list of not visited cities decide which city visit next
            probabilities=[]                                                        #stores the probability for each not visited city
            prob_complement=[]                                                        #complement of above
            
            #chooses next city (trans: starts a decision loop to choose next city (j))
            for j in tabu_list_k:
                  probability_il = 0                                                #sets probability for not visited cities to 0
                  for l in tabu_list_k:                                             #loops over all the cities not visited 
                        probability_il += (pheromones_t[i][l]*alpha*(visibilities[i][l])*beta) #calculates cumulative probability of not visited cities
                        #probability_il=round(probability_il,3)

                  probability_ij = (pheromones_t[i][j]*alpha*(1/distance_func(i,j))*beta) /probability_il #probability of visiting next city given the formula
                  probabilities.append(probability_ij)                           #adds  probability of each city (to become next city "j" ) into a list to later choose randomly from it
                  #prob_complement.append(j)                                        #complement from above
            
            #choose next city
            print(probabilities)
            next_city=random.choices(tabu_list_k, weights = probabilities)       #random.choices function returns a list type, chooses randomly but takes "pheromones into account"
            route_lenght=route_lenght+distance_func(i,j)

            #update pheromones
            pheromones_t*=p
            pheromone_value=pheromones_t[i][next_city]+(1/route_lenght)
            pheromones_t[i][next_city]=pheromone_value

            i=next_city[0]                                                           #new city that is going to be visited
            route_ant_k.append(i)                                                   #adds the city to the route (list)                                                          
            tabu_list_k = np.delete(tabu_list_k,np.where(tabu_list_k==i))           #deletes from the tabu list on this ant k the city they are visiting
            if len(tabu_list_k)==0:                                                 #when all cities on the tabu list have been visited the loop finishes
                  break

      ants_routes.append((route_ant_k,route_lenght))


best=ants_routes[0][1]      
for x in range (len(ants_routes)):
      if ants_routes[x][1]<=best:
            best=ants_routes[x][1]
            index=x
print(ants_routes[index])


[2.1124673415283048e-215, 7.699642667416032e-216, 1.6887089409833872e-214, 7.187877012255966e-216, 9.896479245719716e-216, 6.927656924497364e-216, 1.0278865624389695, 6.163803218332041e-216, 1.3979579596559324e-215, 4.9357429028274725e-216, 5.147606809159612e-216, 6.633170470613307e-214, 1.2032641159274865e-215, 6.765046673163916e-216, 5.183107846924252e-216, 5.103535569292348e-216, 5.086541061810646e-216, 5.0689864227614395e-216, 2.255323305427327e-215, 8.459370302677944e-216, 5.918932523097371e-216, 6.477868211306148e-216, 5.526260676412019e-216, 6.11862092356117e-216, 6.042021690038352e-216, 4.001109858795608e-216, 4.563215921274541e-216, 5.7706265977863896e-216, 5.0735354800786853e-216, 3.406530251093137e-216, 4.5667629746238124e-216, 3.563940279319523e-216, 3.510315324754788e-216, 4.0775834056980105e-216, 3.298542313422509e-216, 3.760612312213069e-216, 3.2883524771502914e-216]
[1.634325127396083e-214, 5.764403883173727e-217, 4.324205504097947e-217, 6.81366346057021e-217, 9.2199858

In [5]:
x=37*38
print(x)

import random

mylist = ["apple", "banana", "cherry"]
print(random.choices(mylist, weights = [10, 1, 1]))


1406
['apple']


BIO
MOHAMED Mohsin, AJANTH Thangavelu
On Embarrassingly Parallel MaxMin Ant Colony Optimization for
Traveling Salesperson Problem. 2021


https://www.diva-portal.org/smash/get/diva2:1610275/FULLTEXT01.pdf
https://iopscience.iop.org/article/10.1088/1742-6596/1566/1/012045/pdf